# Python Movie Process Modules
### Ver. 1.0
### noguhiro2002, 2022/02/20

---

## Python Functions

In [2]:
def inputExcelToProcessList(excelInput_path):
    import pandas as pd
    import numpy as np
    import pprint

    #import excel file
    df = pd.read_excel(excelInput_path)
    df = df.fillna("")

    # extract "processState = 1_submit"
    df_onlySubmit=df.query('processState == "1_submit"')

    # Movie Title groupby
    df_onlySubmitMovieTitleGroupby=df_onlySubmit.groupby("MovieTitle")

    # Set inputMoviePathBase
    inputMoviePathBase="./GoProRawVideos/{}/100GOPRO/{}"

    #各タイトル共通のものを抽出し、それぞれにFor
    MovieTitleList=[]
    RawMoviePathList=[]
    deleteTimeList=[]
    for MovieTitle_str in df_onlySubmitMovieTitleGroupby.groups:
        # Add MovieTitleList
        MovieTitleList.append(MovieTitle_str)

        df_onlySubmitMovieTitle=df_onlySubmit.query('MovieTitle == @MovieTitle_str')

        ## exception: microSDName
        for microSDName_str in df_onlySubmitMovieTitle.groupby("microSDName").groups:
            df_onlySubmitMovieTitleMicroSD=df_onlySubmitMovieTitle.query('microSDName == @microSDName_str')

            RawMoviePath_tmpList=[]
            deleteTime_tmp0List=[]
            ## exception: microSDName AND RawMovieFileName
            for RawMovieFileName_str in df_onlySubmitMovieTitleMicroSD.groupby("RawMovieFileName").groups:
                df_onlySubmitMovieTitleMicroSDRawMovieFileName=df_onlySubmitMovieTitleMicroSD.query('RawMovieFileName == @RawMovieFileName_str')

                ## Add InputMoviePathList
                RawMoviePath_tmpList.append(inputMoviePathBase.format(microSDName_str, RawMovieFileName_str))

                ## Add deleteTimeList
                df_onlySubmitMovieTitleMicroSDRawMovieFileNameTrimming=df_onlySubmitMovieTitleMicroSDRawMovieFileName.query('Action == "Trimming"')
                df_onlySubmitMovieTitleMicroSDRawMovieFileNameNoTrimming=df_onlySubmitMovieTitleMicroSDRawMovieFileName.query('Action == ""')

                ### If Trimming
                if df_onlySubmitMovieTitleMicroSDRawMovieFileNameNoTrimming.empty:
                    deleteTime_tmp1List=[]
                    for NumDeleteTime in range(len(df_onlySubmitMovieTitleMicroSDRawMovieFileNameTrimming)):
                        editStartTime=df_onlySubmitMovieTitleMicroSDRawMovieFileNameTrimming["editStartTime"].values[NumDeleteTime]
                        editEndTime=df_onlySubmitMovieTitleMicroSDRawMovieFileNameTrimming["editEndTime"].values[NumDeleteTime]
                        deleteTime_tmp1List.append([str(editStartTime), str(editEndTime)])

                    deleteTime_tmp0List.append(deleteTime_tmp1List)

                else:
                    deleteTime_tmp0List.append([])

        RawMoviePathList.append(RawMoviePath_tmpList)
        deleteTimeList.append(deleteTime_tmp0List)

    return(MovieTitleList, RawMoviePathList, deleteTimeList)


## Add time stamp and water mark
## Add movie delete (trimming) function
def rawMovieEdit(inputVideo_path, outputVideo_path, deleteTimeList, watermarkPath):
    import cv2
    import ffmpeg
    import datetime
    import tqdm
    import numpy as np

    # print("---start---")

    # Load video via CV2
    video = cv2.VideoCapture(inputVideo_path)

    # WaterMark picture import
    mark = cv2.imread(watermarkPath)  # Water mark picture

    # Water mark settings
    watermark_top_right_x, watermark_top_right_y = 20, 20  # Location of watermark picture (loc of right-top to right-top(watermark)).
    percent_of_scaling = 20

    # Get creation date from load movie
    createDT_str=ffmpeg.probe(inputVideo_path)["format"]["tags"]["creation_time"]
    createDT_date=datetime.datetime.strptime(createDT_str, '%Y-%m-%dT%H:%M:%S.%fZ')

    # get time Delete video time
    def strToTime(x):
        # return datetime.datetime.strptime(str(x), '%H:%M:%S')
        hours, minutes, seconds = map(int, x.split(":"))
        return datetime.timedelta(hours=hours, minutes=minutes, seconds=seconds)

    deleteTimeList_timedelta = []
    for i in deleteTimeList:
        # print(i)
        deleteTimeList_timedelta.append(list(map(strToTime, i)))

    # set Trimming period
    periodList_len=len(deleteTimeList_timedelta) - 1 
    print("trimming period num: ", periodList_len)


    # Get size of width, height
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    size = (width, height)
    print("Size: ", str(size))

    # Resize watermark image and calculate watermark location
    resize_width = int(mark.shape[1] * percent_of_scaling/100)
    resize_height = int(mark.shape[0] * percent_of_scaling/100)

    watermark_x = width - watermark_top_right_x - resize_width
    watermark_y = watermark_top_right_y

    resized_mark = cv2.resize(mark, (resize_width, resize_height), interpolation=cv2.INTER_AREA)
    roi = np.s_[watermark_y : watermark_y + resized_mark.shape[0], watermark_x : watermark_x + resized_mark.shape[1]]  # 編集する領域


    # Get full frame number of loaded movie
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print("All Frame: ", frame_count)

    # Get framerate (milli second)
    frame_rate = 60
    print("Frame rate: ", frame_rate)

    # save
    fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    writer = cv2.VideoWriter(outputVideo_path, fmt, frame_rate, size)

    n=0
    for i in tqdm.tqdm(range(frame_count)):
        ret, frame = video.read()
        ### 加工処理, スタート ###
        # Get millisecond and trans it to hh:mm:ss.mm
        timestamp = video.get(cv2.CAP_PROP_POS_MSEC)
        td = datetime.timedelta(milliseconds=timestamp)
        # Put current DateTime on each frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        # cv2.putText(img,str(datetime.now()),(10,30), font, 1,(255,255,255),2,cv2.LINE_AA)
        writeDateTime=createDT_date + td
        cv2.putText(frame,str(writeDateTime),(50,50), font, 1,(255,255,255),2,cv2.LINE_AA)
        #Add watermark
        # Do Alpha blend
        frame[roi] = cv2.addWeighted(frame[roi], 1, resized_mark, 0.3, 0)
        # Trimming and writing
        if n > periodList_len:
            writer.write(frame)
        elif deleteTimeList_timedelta[n][0] <= td <= deleteTimeList_timedelta[n][1]:
            pass
        elif deleteTimeList_timedelta[n][1] < td:
            writer.write(frame)
            n = n + 1 
        else:
            writer.write(frame)
        ### 加工処理, エンド ###

    writer.release()
    video.release()
    cv2.destroyAllWindows()

    # print("---end---")

#############

## encode movie merge
def movieMerge(videoPathList, outputVideo_path):
    import ffmpeg

    print(videoPathList)

    with open("./tmp.txt", "w") as fp:
        lines = [f"file '{line}'" for line in videoPathList] # file path.
        fp.write("\n".join(lines))
    # Merge ffmpeg (no-encode)
    ffmpeg.input("./tmp.txt", f="concat", safe=0).output(outputVideo_path, c="copy").run(overwrite_output=True, quiet=True)
    return(0)

###########

## encode movie with ffmpeg

def videoEncode(inputVideoPath, outputVideoPath, outputBitrate):
    import ffmpeg

    video=ffmpeg.input(inputVideoPath) 
    video=video.output(outputVideoPath,video_bitrate=outputBitrate*1000, format="mp4") #Encode of the movie
    video=video.run(overwrite_output=True, quiet=True) #Overwrite the movie

###########

## Run: Auto Movie Process

In [3]:
import os
import tqdm


watermarkPath="./watermark/WatermarkSquare.png"
inputProcessExcel="./AutoMovieEditDB.xlsx"

encodeBitrate=8000 #kbps
tmpDirPath="./tmp/"


MovieTitleList, RawMoviePathList, deleteTimeList = inputExcelToProcessList(inputProcessExcel)

for i in range(len(MovieTitleList)):
    print("################")
    print("Processing: ", MovieTitleList[i])
    print("RawMoviePathList: ", RawMoviePathList[i])
    print("deleteTimeList: ", deleteTimeList[i])
    
    ## Add Timestamp and Watermark (&Trimming) for every movies
    mergeVideoInputPathList=[]
    for num, inputVideo_path in enumerate(RawMoviePathList[i]):
        outputVideo_path=tmpDirPath + MovieTitleList[i] + "_" +str(num) + "_rawMovieEdited.mp4"
        mergeVideoInputPathList.append(outputVideo_path)
        # print(deleteTimeList[i][num])
        rawMovieEdit(inputVideo_path, outputVideo_path, deleteTimeList[i][num], watermarkPath)

    ## Merge and encode Movies
    if 1 == len(mergeVideoInputPathList):
        mergeVideoOutputPath=outputVideo_path
        # pass
    else:
        mergeVideoOutputPath=tmpDirPath + MovieTitleList[i] + "_merged.mp4"
        movieMerge(mergeVideoInputPathList, mergeVideoOutputPath)
        
    ## Encode Movie
    encodeInputVideoPath=mergeVideoOutputPath
    encodeOutputVideoPath=tmpDirPath + MovieTitleList[i] + "_final.mp4"
    videoEncode(encodeInputVideoPath, encodeOutputVideoPath, encodeBitrate)
    
    ## Delete Files If successed all process
    if os.path.exists(encodeOutputVideoPath):
        # remove _rawMovieEdited.mp4
        for num, i in enumerate(mergeVideoInputPathList):
            os.remove(i)
        # remove _merged.mp4
        if 1 == len(mergeVideoInputPathList):
            pass
        else:
            os.remove(mergeVideoOutputPath)    

################
Processing:  TrimmingAndEncode
RawMoviePathList:  ['./GoProRawVideos/microSD_No1/100GOPRO/GX010042.MP4']
deleteTimeList:  [[['00:00:03', '00:00:05'], ['00:00:06', '00:00:07'], ['00:00:08', '00:00:09']]]
trimming period num:  2
Size:  (1920, 1080)
All Frame:  587
Frame rate:  60


100%|████████████████████████████████████████████████████████████████████| 587/587 [00:07<00:00, 77.43it/s]


################
Processing:  TrimmingAndMergeAndEncode
RawMoviePathList:  ['./GoProRawVideos/microSD_No1/100GOPRO/GX010042.MP4', './GoProRawVideos/microSD_No1/100GOPRO/GX010043.MP4']
deleteTimeList:  [[], [['00:00:03', '00:00:05'], ['00:00:06', '00:00:07'], ['00:00:08', '00:00:09']]]
trimming period num:  -1
Size:  (1920, 1080)
All Frame:  587
Frame rate:  60


100%|████████████████████████████████████████████████████████████████████| 587/587 [00:09<00:00, 62.79it/s]


trimming period num:  2
Size:  (1920, 1080)
All Frame:  587
Frame rate:  60


100%|████████████████████████████████████████████████████████████████████| 587/587 [00:07<00:00, 82.79it/s]


['./tmp/TrimmingAndMergeAndEncode_0_rawMovieEdited.mp4', './tmp/TrimmingAndMergeAndEncode_1_rawMovieEdited.mp4']
################
Processing:  onlyEncode
RawMoviePathList:  ['./GoProRawVideos/microSD_No1/100GOPRO/GX010042.MP4']
deleteTimeList:  [[]]
trimming period num:  -1
Size:  (1920, 1080)
All Frame:  587
Frame rate:  60


100%|████████████████████████████████████████████████████████████████████| 587/587 [00:09<00:00, 63.46it/s]
